In [101]:
from binary_classifier2 import *
import string

### Character Based

In [3]:
data = load_csv_data(['data/short_excerpts_df.csv'])

In [4]:
from sklearn.model_selection import train_test_split  
data_train, data_test = train_test_split(pd.DataFrame(data),
                    test_size=0.2, random_state=0)

In [135]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline

#vec = CountVectorizer()
#vec = TfidfVectorizer(stop_words='english')
vec = TfidfVectorizer(min_df=5, max_df=0.9, analyzer='char_wb', ngram_range=(3,10))
clf = LogisticRegressionCV(class_weight='balanced')
#clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(data_train.Excerpts, data_train.ACCOUNT)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=5,
        ngram_range=(3, 10), norm='l2', preprocessor=None, smoo...2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

In [54]:
from sklearn import metrics

def print_report(pipe):
    y_test = data_test.ACCOUNT
    y_pred = pipe.predict(data_test.Excerpts)
    report = metrics.classification_report(y_test, y_pred)#, target_names=twenty_test.target_names)
    print(metrics.confusion_matrix(y_test, y_pred))
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe)

[[7006  858]
 [ 408  813]]
             precision    recall  f1-score   support

          0       0.94      0.89      0.92      7864
          1       0.49      0.67      0.56      1221

avg / total       0.88      0.86      0.87      9085

accuracy: 0.861


In [134]:
import eli5
print_report(pipe)
eli5.show_weights(clf, vec= vec, top=20)

[[7155  709]
 [ 474  747]]
             precision    recall  f1-score   support

          0       0.94      0.91      0.92      7864
          1       0.51      0.61      0.56      1221

avg / total       0.88      0.87      0.87      9085

accuracy: 0.870


Weight?,Feature
+19.295,s?
+17.027,e?
+15.234,a?
+14.467,nace
+13.305,fame
+12.336,ak
+11.586,es?
+11.561,why
+11.504,"t,�"
+11.504,"t,�"


In [34]:
import warnings
warnings.filterwarnings('ignore')
doc=list(data_test.Excerpts)[1]
eli5.show_prediction(clf, doc, vec=vec)

### Dimension Reduction

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, make_pipeline

vec = TfidfVectorizer(min_df=5, stop_words='english',
                      ngram_range=(1, 3))
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
lsa = make_pipeline(vec, svd)

#clf = SVC(C=150, gamma=2e-2, probability=True, kernel = 'linear', class_weight = 'balanced')
clf = SVC(kernel = 'linear', class_weight = 'balanced')
pipe = make_pipeline(lsa, clf)
pipe.fit(data_train.Excerpts, data_train.ACCOUNT)
pipe.score(data_test.Excerpts, data_test.ACCOUNT)

0.7002751788662631

In [57]:
print_report(pipe)

[[5383 2481]
 [ 242  979]]
             precision    recall  f1-score   support

          0       0.96      0.68      0.80      7864
          1       0.28      0.80      0.42      1221

avg / total       0.87      0.70      0.75      9085

accuracy: 0.700


In [58]:
eli5.show_weights(clf, vec= vec, top=20)

Weight?,Feature
+13.845,100 times
+12.543,10 39 minute
+10.913,10 years
+9.981,10
+8.749,100 times 000
+8.362,100 people
+8.219,000 people 60
+8.171,12 crime scenes
+7.675,11 days
+7.540,11 2001


In [41]:
from eli5.lime import TextExplainer
doc=list(data_test.Excerpts)[1]
te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction()

### Pre-processing with BERT method

In [71]:
import bert
from bert import tokenization
import tensorflow as tf
import tensorflow_hub as hub

In [76]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                tokenization_info["do_lower_case"]])

    return bert.tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

W0723 13:21:53.989967 4620117440 deprecation_wrapper.py:119] From /Users/andreapagotto/anaconda/lib/python3.5/site-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [93]:
toks = tokenizer.tokenize("This here's an example of using the BERT tokenizer. There were 1000 people.")
[tok for tok in toks if not tok.isnumeric()]

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer',
 '.',
 'there',
 'were',
 'people',
 '.']

In [107]:
# create custom tokenizer with additional pre-processing on BERT tokenizer
tokenizer = create_tokenizer_from_hub_module()
def create_custom_tokenizer(tokenizer=tokenizer, punct = True, numbers=False):
    def custom_tokenizer(text):
        tok_text = tokenizer.tokenize(text)
        if not numbers:
            tok_text = [tok for tok in tok_text if not tok.isnumeric()]
        if not punct:
            tok_text = [tok for tok in tok_text if not all(c in string.punctuation for c in tok)]
        return tok_text
    return custom_tokenizer

cust_tok_no_punct = create_custom_tokenizer(tokenizer=tokenizer, punct = False, numbers=False)
cust_tok_no_punct("This here's an example of using the BERT tokenizer. There were 1000 people.")

['this',
 'here',
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer',
 'there',
 'were',
 'people']

In [108]:
vec = TfidfVectorizer(tokenizer=cust_tok_no_punct, ngram_range=(1, 3), min_df = 5)
clf = LogisticRegressionCV(class_weight='balanced')
#clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(data_train.Excerpts, data_train.ACCOUNT)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_i...2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

In [109]:
print_report(pipe)
eli5.show_weights(clf, vec= vec, top=20)

[[7298  566]
 [ 455  766]]
             precision    recall  f1-score   support

          0       0.94      0.93      0.93      7864
          1       0.58      0.63      0.60      1221

avg / total       0.89      0.89      0.89      9085

accuracy: 0.888


Weight?,Feature
+10.047,blame
+8.668,women
+8.179,motives
+8.000,culture
+7.780,legally
+7.594,in other
+7.566,ms salman
+7.410,blamed
+7.151,question
+6.898,for it


In [110]:
vec = TfidfVectorizer(tokenizer=cust_tok_no_punct, ngram_range=(1, 3), min_df = 5)
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(data_train.Excerpts, data_train.ACCOUNT)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_i...2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

In [111]:
print_report(pipe)
eli5.show_weights(clf, vec= vec, top=20)

[[7587  277]
 [ 616  605]]
             precision    recall  f1-score   support

          0       0.92      0.96      0.94      7864
          1       0.69      0.50      0.58      1221

avg / total       0.89      0.90      0.89      9085

accuracy: 0.902


Weight?,Feature
+9.211,blame
+7.954,women
+7.447,motives
+7.037,culture
+6.719,legally
+6.546,blamed
+6.510,ms salman
+6.498,in other
+6.355,violence
+6.354,question


## Test Combinations of Preprocessing

In [116]:
# create different tokenizers:
cust_tok_only_alpha = create_custom_tokenizer(tokenizer=tokenizer, punct = False, numbers=False)
cust_tok_no_nums = create_custom_tokenizer(tokenizer=tokenizer, punct = True, numbers=False)
cust_tok_all_chars = create_custom_tokenizer(tokenizer=tokenizer, punct = True, numbers=True)

cust_tokenizers_dict = {'cust_all':cust_tok_all_chars, 
                       'cust_no_nums':cust_tok_no_nums,
                        'cust_only_alpha':cust_tok_only_alpha}

In [117]:
# create custom tokenizer vectorizers:
vectorizers_dict = {}

for cust_tok_name in cust_tokenizers_dict.keys():
    cust_tokenizer = cust_tokenizers_dict[cust_tok_name]
    
    cust_vectorizer = TfidfVectorizer(tokenizer=cust_tokenizer, ngram_range=(1, 3), min_df = 10, max_df=0.9)
    vec_name = cust_tok_name+"-3gram"
    vectorizers_dict[vec_name]=cust_vectorizer
    
    cust_vectorizer= TfidfVectorizer(tokenizer=cust_tokenizer, ngram_range=(1, 1), min_df = 10, max_df=0.9)
    vec_name = cust_tok_name+"-1gram"
    vectorizers_dict[vec_name]=cust_vectorizer
        

In [118]:
# add vectorizers with regular, and character based pre-processing
vectorizers_dict['3gram'] = TfidfVectorizer(ngram_range=(1, 3), min_df = 10, max_df=0.9)
vectorizers_dict['1gram'] = TfidfVectorizer(ngram_range=(1, 1), min_df = 10, max_df=0.9)
vectorizers_dict['char'] = TfidfVectorizer(ngram_range=(3, 10), min_df = 10, max_df=0.9, analyzer='char')

In [129]:
clf = LogisticRegressionCV()
results = pd.DataFrame(columns = ['vectorizer', 'precision', 'recall', 'fscore'])

for vec_name in vectorizers_dict.keys():
    # train
    vec = vectorizers_dict[vec_name]
    pipe = make_pipeline(vec, clf)
    pipe.fit(data_train.Excerpts, data_train.ACCOUNT)
    
    # test
    y_test = data_test.ACCOUNT
    y_pred = pipe.predict(data_test.Excerpts)
    precision, recall, fscore, support\
    = metrics.precision_recall_fscore_support(y_test, y_pred, average='binary', pos_label=1)
    
    # save results
    results_row = [vec_name, precision, recall, fscore]
    print(results_row)
    results = results.append(results_row)

['cust_all-1gram', 0.7573415765069552, 0.4013104013104013, 0.5246252676659529]
['char', 0.6811428571428572, 0.4881244881244881, 0.5687022900763359]
['cust_all-3gram', 0.7990867579908676, 0.42997542997543, 0.5591054313099042]
['1gram', 0.7343511450381679, 0.3939393939393939, 0.5127931769722814]
['cust_no_nums-3gram', 0.8015503875968992, 0.42342342342342343, 0.5541264737406216]
['cust_only_alpha-3gram', 0.7948328267477204, 0.4283374283374283, 0.5566790846194785]
['3gram', 0.7794336810730254, 0.4283374283374283, 0.5528541226215644]
['cust_only_alpha-1gram', 0.743202416918429, 0.40294840294840295, 0.5225703664365375]
['cust_no_nums-1gram', 0.7515432098765432, 0.3988533988533989, 0.5211342964151953]


In [132]:
clf = LogisticRegressionCV(class_weight='balanced')
results_balanced = pd.DataFrame(columns = ['vectorizer', 'precision', 'recall', 'fscore'])

for vec_name in vectorizers_dict.keys():
    # train
    vec = vectorizers_dict[vec_name]
    pipe = make_pipeline(vec, clf)
    pipe.fit(data_train.Excerpts, data_train.ACCOUNT)
    
    # test
    y_test = data_test.ACCOUNT
    y_pred = pipe.predict(data_test.Excerpts)
    precision, recall, fscore, support\
    = metrics.precision_recall_fscore_support(y_test, y_pred, average='binary', pos_label=1)
    
    # save results
    results_row = [vec_name, precision, recall, fscore]
    print(results_row)
    results_balanced = results_balanced.append(results_row)

['cust_all-1gram', 0.4458666666666667, 0.6846846846846847, 0.5400516795865633]
['char', 0.5923202614379085, 0.5937755937755937, 0.5930470347648262]
['cust_all-3gram', 0.5462555066079295, 0.6093366093366094, 0.5760743321718932]
['1gram', 0.44461305007587254, 0.7199017199017199, 0.5497185741088181]
['cust_no_nums-3gram', 0.5461936437546193, 0.6052416052416052, 0.5742035742035742]
['cust_only_alpha-3gram', 0.5327540106951871, 0.6527436527436528, 0.5866764814133235]
['3gram', 0.5242782152230971, 0.6543816543816544, 0.5821493624772315]
['cust_only_alpha-1gram', 0.4417989417989418, 0.6838656838656839, 0.5368048858887818]
['cust_no_nums-1gram', 0.4407684098185699, 0.6764946764946765, 0.5337641357027464]


In [138]:
def run_classifiers(vectorizers_dict, classifiers_dict):
    results = pd.DataFrame(columns = ['classifier','vectorizer', 'precision', 'recall', 'fscore'])
    
    for classifier_name in classifiers_dict.keys():
        clf = classifiers_dict[classifier_name]
        for vec_name in vectorizers_dict.keys():
            # train
            vec = vectorizers_dict[vec_name]
            pipe = make_pipeline(vec, clf)
            pipe.fit(data_train.Excerpts, data_train.ACCOUNT)

            # test
            y_test = data_test.ACCOUNT
            y_pred = pipe.predict(data_test.Excerpts)
            precision, recall, fscore, support\
            = metrics.precision_recall_fscore_support(y_test, y_pred, average='binary', pos_label=1)

            # save results
            results_row = [classifier_name, vec_name, precision, recall, fscore]
            print(results_row)
            results = results.append(results_row)
        
    return results

In [139]:
logregb = LogisticRegressionCV(class_weight='balanced')
logreg = LogisticRegressionCV()
classifiers_dict = {'logregcv_balanced': logregb, 'logreg':logreg}


In [ ]:
run_classifiers(vectorizers_dict, classifiers_dict)

['logreg', 'cust_all-1gram', 0.7573415765069552, 0.4013104013104013, 0.5246252676659529]
